In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.device("cuda:0")

device(type='cuda', index=0)

Hyper params

In [66]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [67]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./mnist data', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./mnist data', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [68]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [69]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [70]:
network = Net()
network.cuda()   # passing arguments to the gpu before we define te optimizer
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

In [71]:
def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        data = data.to('cuda')
        target = target.to('cuda')
        output = network(data)
        loss = F.nll_loss(output, target) # negative log likelihood loss.
        loss.backward() # computes dloss/dx for every parameter x which has requires_grad=True. 
        optimizer.step() # updates the value of x using the gradient x.grad.
    if batch_idx % log_interval == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
        train_losses.append(loss.item())
        train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
        torch.save(network.state_dict(), '/results/model.pth')
        torch.save(optimizer.state_dict(), '/results/optimizer.pth')

In [72]:
def test():
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to('cuda')
            target = target.to('cuda')
            print("data s {} target s {}".format(len(data),len(target)))

            output = network(data)
            print("output s {} target s {}".format(len(output),len(target)))

            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [73]:
test()
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()

data s 1000 target s 1000
output s 1000 target s 1000
data s 1000 target s 1000


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


output s 1000 target s 1000
data s 1000 target s 1000
output s 1000 target s 1000
data s 1000 target s 1000
output s 1000 target s 1000
data s 1000 target s 1000
output s 1000 target s 1000
data s 1000 target s 1000
output s 1000 target s 1000
data s 1000 target s 1000
output s 1000 target s 1000
data s 1000 target s 1000
output s 1000 target s 1000
data s 1000 target s 1000
output s 1000 target s 1000
data s 1000 target s 1000
output s 1000 target s 1000

Test set: Avg. loss: 2.3316, Accuracy: 1137/10000 (11%)

data s 1000 target s 1000
output s 1000 target s 1000
data s 1000 target s 1000
output s 1000 target s 1000
data s 1000 target s 1000
output s 1000 target s 1000
data s 1000 target s 1000
output s 1000 target s 1000
data s 1000 target s 1000
output s 1000 target s 1000
data s 1000 target s 1000
output s 1000 target s 1000
data s 1000 target s 1000
output s 1000 target s 1000
data s 1000 target s 1000
output s 1000 target s 1000
data s 1000 target s 1000
output s 1000 target s 1